In [35]:
import pymysql
import pandas as pd
import random

In [16]:
db = pymysql.connect(
    user='root', 
    port = 3306,
    password='zxcv0504', 
    host='127.0.0.1', 
    charset='utf8',
    db = 'project'
)

In [22]:
def Login():
    cursor = db.cursor()
    while True:
        user_name = input("아이디를 입력하세요.")
        sql = ''' 
        SELECT * 
        FROM user
        WHERE UserName = "''' + user_name + '";'
        check = cursor.execute(sql)
        if check == 0:
            print("해당하는 아이디가 없습니다. 다시 입력해주세요.")
            continue
        password = input("비밀번호를 입력하세요.")
        tmp = cursor.fetchall()[0]
        if password == tmp[2]:
            print("로그인 성공! 아이디 :", user_name)
            break
        else: 
            print("비밀번호가 틀렸습니다. 다시 입력해주세요.")
    return tmp[0]


In [ ]:
# (Login ex) 선우준 bjiurf

In [24]:
Login()

아이디를 입력하세요.신세은
비밀번호를 입력하세요.looemm
로그인 성공! 아이디 : 신세은


5

In [15]:
def Sign_up():
    cursor = db.cursor()
    while True:
        user_name = input("아이디를 입력하세요.")
        sql = ''' 
        SELECT * 
        FROM user
        WHERE UserName = "''' + user_name + '";'
        check = cursor.execute(sql)
        if check == 1:
            print("중복된 아이디입니다. 다른 아이디를 입력해주세요.")
            continue
        else:
            break
            
    password = input("비밀번호를 입력하세요.")
    age = input("나이를 입력하세요.")
    sex = input("성별(m 또는 f)를 입력하세요.")
    if sex == "m": sex = 0
    else: sex = 1

    print("회원가입이 완료되었습니다!")
    n = cursor.execute("SELECT * FROM USER")
    cursor.execute("INSERT INTO USER VALUES(%d, '%s', '%s', '%s', '%s');"%(n, user_name, password, age, sex))
    db.commit()
    
    f = pd.read_csv("Data/User Data.csv")
    f.loc[len(f)] = [n, user_name, password, age, sex]
    f.to_csv("Data/User Data.csv", index = False)
    return n

In [17]:
Sign_up()

아이디를 입력하세요.홍길동
비밀번호를 입력하세요.1234
나이를 입력하세요.20
성별(m 또는 f)를 입력하세요.m
회원가입이 완료되었습니다!


20

In [100]:
def Recommendation(user_ID, n):
    cursor = db.cursor()
    # 장르별 추천
    if n == 1: 
        sql = """
        SELECT Genre, count(Genre) FROM user_has_movie u INNER JOIN genre_has_movie m ON m.Movie_Code = u.Movie_Code 
        WHERE User_ID = %d
        GROUP BY genre
        ORDER BY count(Genre) DESC;
        """%user_ID
        cursor.execute(sql)
        genre = cursor.fetchall()[0][0]
        
        sql = """
        SELECT Title, Genre
        FROM movie m JOIN genre_has_movie g ON m.Movie_Code = g.Movie_code JOIN rating r ON r.Movie_code = m.Movie_code
        WHERE Genre = "%s"
        ORDER BY (((`r`.`Rating_Naver` * `r`.`Num_Naver`) + (`r`.`Rating_Daum` * `r`.`NUM_Daum`)) / (`r`.`Num_Naver` + `r`.`NUM_Daum`)) DESC
        LIMIT 5;"""%genre
        
        cursor.execute(sql)
        print("회원님이 가장 즐겨보시는 장르는 %s네요!"%genre)
        print("%s 장르에서 평점이 가장 높은 영화 5개입니다,"%genre)
        for a, i in enumerate(cursor.fetchall()):
            print(a+1, i[0])
    # 연령대별 추천
    elif n == 2:
        sql = """
        SELECT Age FROM user
        WHERE User_ID = %d;"""%user_id
        cursor.execute(sql)
        age = cursor.fetchall()[0][0]
        
        sql = """
        SELECT m.Title, count(m.Movie_Code) c
        FROM user u INNER JOIN user_has_movie um ON u.user_ID = um.user_ID JOIN movie m on um.Movie_Code = m.Movie_Code
        WHERE u.Age BETWEEN %d and %d
        GROUP BY m.Movie_Code
        ORDER BY c DESC
        LIMIT 5;"""%(age//10 * 10, age//10 * 10 + 9)
        cursor.execute(sql)
        print("회원님의 연령대에서 가장 인기가 많은 영화 5개입니다,")
        for a, i in enumerate(cursor.fetchall()):
            print(a+1, i[0])
    elif n == 3:
        cursor.execute("SELECT Title FROM movie;")
        random_movie = random.choice(list(cursor.fetchall()))[0]
        print("이런 영화는 어떠신가요? :", random_movie)

In [92]:
Recommendation(5,2)

회원님의 연령대에서 가장 인기가 많은 영화 5개입니다,
1 물의
2 계절과 계절 사이
3 셧다운
4 극장판 기븐
5 죽지않는 인간들의 밤


In [101]:
def main():
    while True:
        print("다음 중 하나를 선택하세요.")
        print("1. 로그인")
        print("2. 회원가입")
        n = "-1"
        while n not in ["1","2"]:
            n = input()
        n = int(n)
        if n == 1:
            user_ID = Login()
            break
        elif n == 2:
            user_ID = Sign_up()
            break
    print()
    print("다음 중 하나를 선택하세요.")
    print("1. 장르별 추천")
    print("2. 연령대별 추천")
    print("3. 오늘의 랜덤 영화는?")           
    while True:
        n = input()
        if n in ["1","2","3"]:
            Recommendation(user_ID, int(n))
            break

In [102]:
main()

다음 중 하나를 선택하세요.
1. 로그인
2. 회원가입
1
아이디를 입력하세요.신세은
비밀번호를 입력하세요.looemm
로그인 성공! 아이디 : 신세은

다음 중 하나를 선택하세요.
1. 장르별 추천
2. 연령대별 추천
3. 오늘의 랜덤 영화는?
1
회원님이 가장 즐겨보시는 장르는 드라마네요!
드라마 장르에서 평점이 가장 높은 영화 5개입니다,
1 물의
2 사선의 끝
3 나의이름
4 오늘도 위위
5 여수 밤바다


In [106]:
main()

다음 중 하나를 선택하세요.
1. 로그인
2. 회원가입
1
아이디를 입력하세요.신세으
해당하는 아이디가 없습니다. 다시 입력해주세요.
아이디를 입력하세요.신세은
비밀번호를 입력하세요.looemm
로그인 성공! 아이디 : 신세은

다음 중 하나를 선택하세요.
1. 장르별 추천
2. 연령대별 추천
3. 오늘의 랜덤 영화는?
3
이런 영화는 어떠신가요? : 소울


In [ ]:
아이디를 입력하세요.신세은
비밀번호를 입력하세요.looemm
로그인 성공! 아이디 : 신세은